In [20]:
from pycaret.classification import *

from sklearn.linear_model import LogisticRegression

from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

In [26]:
train = pd.read_csv('../data/fe_train2.csv')
test = pd.read_csv('../data/fe_test2.csv')


raw_test = pd.read_csv('../data/test.csv')

In [3]:
train.head()

,Aged,CyberProtection,TechnicalAssistance,SettlementProcess,QuarterlyPayment,CustomerAttrition,GP / QP,GP / ServiceSpan,QP / ServiceSpan,aged_married,aged_dependents,married_dependents,cyber_tech,ss_cluster,gp_cluster
0,0,0,1,2,96.259802,1.0,48.079562,94.451615,1.964486,0,1,1,1,0,1
1,0,0,0,0,18.876470,0.0,48.848434,20.045348,0.410358,1,0,2,0,0,0
2,1,1,1,1,106.016057,1.0,58.537275,103.431519,1.766934,3,2,2,3,2,1
3,0,0,0,2,80.108839,0.0,15.586987,83.243693,5.340589,0,0,0,0,1,0
4,0,0,0,2,51.529773,0.0,24.943274,49.435434,1.981914,1,1,3,0,3,0


In [6]:
ig_feats = ['ID']
num_feats = test.columns.tolist()

data_setup = setup(
    data=train,
    target='CustomerAttrition',
    train_size=0.75,
#     ignore_features=ig_feats,
#     imputation_type='iterative',
#     numerical_imputation='',
    preprocess=False,
    fold_shuffle=True,
    
)

,Description,Value
0,session_id,8083
1,Target,CustomerAttrition
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(6338, 15)"
5,Missing Values,False
6,Numeric Features,4
7,Categorical Features,10
8,Transformed Train Set,"(4753, 14)"
9,Transformed Test Set,"(1585, 14)"


In [8]:
best_models = compare_models(n_select=3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.7902,0.8304,0.4829,0.6554,0.5547,0.4218,0.4310,0.3220
catboost,CatBoost Classifier,0.7862,0.8249,0.4884,0.6393,0.5523,0.4155,0.4228,2.1520
ada,Ada Boost Classifier,0.7858,0.8230,0.4829,0.6401,0.5491,0.4126,0.4204,0.3380
lr,Logistic Regression,0.7837,0.8251,0.4604,0.6416,0.5348,0.3992,0.4092,0.1590
lda,Linear Discriminant Analysis,0.7837,0.8231,0.4674,0.6387,0.5382,0.4018,0.4110,0.0230
ridge,Ridge Classifier,0.7801,0.0000,0.4115,0.6497,0.5026,0.3709,0.3873,0.0200
rf,Random Forest Classifier,0.7757,0.8053,0.4635,0.6147,0.5271,0.3843,0.3916,0.3400
nb,Naive Bayes,0.7597,0.7790,0.5979,0.5529,0.5739,0.4071,0.4082,0.0210
knn,K Neighbors Classifier,0.7585,0.7627,0.4565,0.5666,0.5046,0.3478,0.3518,0.0390
et,Extra Trees Classifier,0.7559,0.7810,0.4542,0.5608,0.5010,0.3422,0.3459,0.2490


In [9]:
gbc = create_model('gbc')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7773,0.8204,0.4961,0.6095,0.5470,0.4014,0.4052
1,0.7584,0.8149,0.4651,0.5660,0.5106,0.3523,0.3553
2,0.8088,0.8357,0.5426,0.6863,0.6061,0.4821,0.4879
3,0.8021,0.8270,0.4496,0.7160,0.5524,0.4337,0.4531
4,0.7916,0.8332,0.4806,0.6596,0.5561,0.4242,0.4333
5,0.7853,0.8350,0.4729,0.6421,0.5446,0.4084,0.4165
6,0.8000,0.8449,0.4884,0.6848,0.5701,0.4445,0.4553
7,0.7874,0.8257,0.4496,0.6591,0.5346,0.4031,0.4155
8,0.8000,0.8407,0.5547,0.6514,0.5992,0.4671,0.4697
9,0.7916,0.8263,0.4297,0.6790,0.5263,0.4013,0.4185


In [10]:
cat = create_model('catboost')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7794,0.8115,0.4961,0.6154,0.5494,0.4055,0.4097
1,0.7731,0.8115,0.4884,0.6000,0.5385,0.3901,0.3938
2,0.8088,0.8296,0.5426,0.6863,0.6061,0.4821,0.4879
3,0.7916,0.8172,0.4496,0.6744,0.5395,0.4117,0.4259
4,0.7832,0.8311,0.4961,0.6275,0.5541,0.4134,0.4184
5,0.7768,0.8313,0.4651,0.6186,0.5310,0.3884,0.3952
6,0.8021,0.8429,0.5116,0.6804,0.5841,0.4576,0.4656
7,0.7768,0.8269,0.4264,0.6322,0.5093,0.3718,0.3839
8,0.7874,0.8407,0.5781,0.6116,0.5944,0.4505,0.4508
9,0.7832,0.8066,0.4297,0.6471,0.5164,0.3839,0.3973


In [11]:
ada = create_model('ada')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7794,0.8217,0.5116,0.6111,0.5570,0.4116,0.4145
1,0.7626,0.8037,0.4729,0.5755,0.5191,0.3635,0.3666
2,0.7899,0.8281,0.5349,0.6330,0.5798,0.4411,0.4439
3,0.7811,0.8037,0.4264,0.6471,0.5140,0.3803,0.3941
4,0.7832,0.8279,0.4496,0.6444,0.5297,0.3945,0.4053
5,0.7853,0.8396,0.5039,0.6311,0.5603,0.4206,0.4253
6,0.8042,0.8320,0.5116,0.6875,0.5867,0.4620,0.4706
7,0.8000,0.8311,0.4574,0.7024,0.5540,0.4324,0.4489
8,0.8021,0.8281,0.5547,0.6574,0.6017,0.4713,0.4743
9,0.7705,0.8139,0.4062,0.6118,0.4883,0.3480,0.3602


In [12]:
tuned_gbc = tune_model(gbc)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7836,0.8173,0.5039,0.6250,0.5579,0.4169,0.4211
1,0.7773,0.7995,0.4961,0.6095,0.5470,0.4014,0.4052
2,0.7983,0.8293,0.5271,0.6602,0.5862,0.4551,0.4601
3,0.7811,0.8192,0.4109,0.6543,0.5048,0.3735,0.3902
4,0.7811,0.8344,0.4806,0.6263,0.5439,0.4031,0.4092
5,0.7895,0.8347,0.5116,0.6408,0.5690,0.4320,0.4368
6,0.7958,0.8362,0.5194,0.6569,0.5801,0.4476,0.4530
7,0.7516,0.8148,0.3953,0.5604,0.4636,0.3082,0.3162
8,0.8000,0.8402,0.5547,0.6514,0.5992,0.4671,0.4697
9,0.7895,0.8051,0.4219,0.6750,0.5192,0.3935,0.4113


In [13]:
tuned_cat = tune_model(cat)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7857,0.8225,0.5194,0.6262,0.5678,0.4270,0.4303
1,0.7878,0.8190,0.5116,0.6346,0.5665,0.4282,0.4325
2,0.7899,0.8281,0.5349,0.6330,0.5798,0.4411,0.4439
3,0.7979,0.8236,0.4496,0.6988,0.5472,0.4249,0.4420
4,0.7937,0.8361,0.4884,0.6632,0.5625,0.4316,0.4402
5,0.7853,0.8391,0.4496,0.6517,0.5321,0.3988,0.4104
6,0.8084,0.8481,0.5194,0.6979,0.5956,0.4736,0.4824
7,0.7811,0.8207,0.4109,0.6543,0.5048,0.3735,0.3902
8,0.7958,0.8356,0.5469,0.6422,0.5907,0.4558,0.4584
9,0.7874,0.8135,0.4141,0.6709,0.5121,0.3857,0.4041


In [14]:
tuned_ada = tune_model(ada)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7794,0.8232,0.4961,0.6154,0.5494,0.4055,0.4097
1,0.7668,0.8068,0.4884,0.5833,0.5316,0.3780,0.3807
2,0.7941,0.8268,0.5116,0.6535,0.5739,0.4408,0.4466
3,0.7853,0.8202,0.4341,0.6588,0.5234,0.3922,0.4065
4,0.7874,0.8353,0.4729,0.6489,0.5471,0.4126,0.4214
5,0.7874,0.8386,0.4729,0.6489,0.5471,0.4126,0.4214
6,0.8126,0.8419,0.5426,0.7000,0.6114,0.4905,0.4974
7,0.7979,0.8330,0.4341,0.7089,0.5385,0.4185,0.4391
8,0.8063,0.8318,0.5547,0.6698,0.6068,0.4798,0.4836
9,0.7895,0.8155,0.4531,0.6591,0.5370,0.4068,0.4187


In [34]:
blended_models = blend_models([gbc, tuned_cat, tuned_ada])

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7794,0.8237,0.4884,0.6176,0.5455,0.4024,0.4073
1,0.7752,0.8172,0.4884,0.6058,0.5408,0.3942,0.3982
2,0.8067,0.8334,0.5581,0.6729,0.6102,0.4832,0.4869
3,0.8000,0.8253,0.4574,0.7024,0.5540,0.4324,0.4489
4,0.7937,0.8361,0.4806,0.6667,0.5586,0.4285,0.4383
5,0.7874,0.8387,0.4806,0.6458,0.5511,0.4157,0.4235
6,0.8063,0.8477,0.5039,0.6989,0.5856,0.4635,0.4741
7,0.7853,0.8248,0.4419,0.6552,0.5278,0.3955,0.4084
8,0.8042,0.8394,0.5781,0.6549,0.6141,0.4836,0.4853
9,0.7937,0.8219,0.4219,0.6923,0.5243,0.4023,0.4224


In [22]:
stacked_models = stack_models([tuned_gbc, tuned_cat, tuned_ada], LogisticRegression())

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7752,0.8191,0.4651,0.6122,0.5286,0.3846,0.3909
1,0.7878,0.8084,0.5039,0.6373,0.5628,0.4252,0.4303
2,0.7878,0.8230,0.4961,0.6400,0.5590,0.4222,0.4281
3,0.7874,0.8279,0.4574,0.6556,0.5388,0.4063,0.4174
4,0.7684,0.8319,0.4419,0.6000,0.5089,0.3619,0.3692
5,0.7937,0.8354,0.4729,0.6703,0.5545,0.4255,0.4364
6,0.8084,0.8390,0.4961,0.7111,0.5845,0.4651,0.4778
7,0.7747,0.7977,0.3953,0.6375,0.4880,0.3537,0.3702
8,0.8000,0.8383,0.5781,0.6435,0.6091,0.4752,0.4764
9,0.7853,0.8133,0.4219,0.6585,0.5143,0.3848,0.4005


In [35]:
final_model = finalize_model(blended_models)

In [36]:
preds = predict_model(final_model, data=test)
preds.head()

,Aged,CyberProtection,TechnicalAssistance,SettlementProcess,QuarterlyPayment,GP / QP,GP / ServiceSpan,QP / ServiceSpan,aged_married,aged_dependents,married_dependents,cyber_tech,ss_cluster,gp_cluster,Label,Score
0,0,0,0,2,21.299937,7.093567,16.788058,2.366660,0,0,0,0,1,2,0.0,0.7753
1,0,0,0,2,56.609623,27.438391,57.528777,2.096653,0,0,0,0,3,0,0.0,0.7280
2,0,0,0,1,25.167690,66.921613,25.138245,0.375637,0,0,0,0,2,0,0.0,0.8247
3,1,0,1,0,55.560968,32.934532,57.183577,1.736280,2,2,0,1,3,0,0.0,0.7280
4,0,0,0,0,20.070002,24.405487,22.264462,0.912273,1,1,3,0,3,2,0.0,0.8240


In [37]:
sub = pd.DataFrame({
    'ID': raw_test['ID'],
    'CustomerAttrition': preds['Label']
})
# sub['CustomerAttrition'] = sub['CustomerAttrition'].astype(int)

sub['CustomerAttrition'] = sub['CustomerAttrition'].replace({'0.0': 'No', '1.0': 'Yes'})
print(sub['CustomerAttrition'].value_counts(normalize=True))
sub.head()

No     0.8
Yes    0.2
Name: CustomerAttrition, dtype: float64


,ID,CustomerAttrition
0,WMORTPCG3HTIPEFSGUFY,No
1,GBLPEA16C3KGBO57KEFO,No
2,IDCQS0CO5P29AMKY3LO1,No
3,1IY29SDITYU6IFZ4KJB0,No
4,T5XMW5B0YPCONH30WJD2,No


In [38]:
sub.to_csv('../submissions/pyc1.csv', index=False)

In [17]:
evaluate_model(best_models)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…